## 2절 플로이드-워셜 최단경로 알고리즘

### 그래프 용어

* 마디 또는 정점(vertex, node)

* 이음선(edge, arc)

* 방향 그래프(directed graph, or digraph)

* 가중치(weight)

* 가중치 포함 그래프(weighted graph)

* 경로(path): 이음선으로 연결된 마디들의 나열. 즉, 하나의 마디에서 다른 마디로 가는 이음선의 연결.

* 단순경로(simple path): 같은 마디를 두 번 지나지 않는 경로

* 순환(cycle): 하나의 마디에서 출발하여 다시 그 마디로 돌아오는 경로

* 순환 그래프(cyclic graph): 순환을 갖는 그래프

* 비순환 그래프 (acyclic graph): 순환을 갖지 않는 그래프

* 경로의 길이(length): 
    * 가중치 포함 그래프의 경우: 경로 상에 있는 가중치의 합
    * 가중치 없는 그래프의 경우: 경로 상에 있는 이음선의 수

### 예제: 가중치 포함 방향그래프

<div align="center"><img src="./images/algo03/algo03-03.png" width="350"/></div>

### 최단경로 문제

* 임의의 하나의 마디에서 다른 임의의 마디로 가는 최단 경로 구하기

* 가중치 포함, 방향성 존중.

* 주의사항: 최단경로는 순환을 포함하지 않아야 함. 즉, 단순경로만 대상으로 삼아도 됨.

#### 예제

* 위 그래프에서 $v_1$에서 $v_3$로 가는 단순경로와 경로 길이:
    * $[v_1, v_2, v_3]$
        * 경로 길이: $1 + 3 = 4$
    * $[v_1, v_4, v_3]$
        * 경로 길이: $1 + 2 = 3$
    * $[v_1, v_2, v_4, v_3]$
        * 경로 길이: $1 + 2 + 2 = 5$        

* 따라서 최단경로와 길이는 다음과 같음.
    * $[v_1, v_4, v_3]$
        * 경로 길이: $1 + 2 = 3$

#### 응용 사례

* 도시 간의 최단 경로

* 다구간 비행기표 여정

* 지도 앱에서 경유 추가

### 최적화 문제

* 하나 이상의 해답 중에서 최적의 값을 갖는 해답을 찾아야 하는 문제

* 최적값: 문제에 따라 최댓값 또는 최솟값을 가리킴.

* 예제: 최단경로 찾기 문제. 
    * 최소 경로길이를 갖는 해답을 찾아야 함.
    * 하나의 마디에서 다른 마디로의 최단경로가 여러 개 있을 수 있음.
    * 그럴 때는 그 중에 하나 선택.

### 최단경로 문제 무작정 알고리즘

* 하나의 마디에서 다른 마디로의 모든 경로의 길이를 계산한 후 그 중에 최소길이 선택.

* 지수보다 나쁜 시간복잡도를 가짐.

#### 무작정 알고리즘 분석

* 가정: 
    * $n$ 개의 마디: $v_1, v_2, ..., v_n$
    * 모든 마디들 사이에 이음선 존재

* $v_1$에서 어떤 마디 $v_n$으로 가는 경로 중 나머지 모든 마디를 한 번씩 꼭 거쳐서 가는 경로들의 수는?
    * $v_1$ 에서 출발하여 처음에 도착할 수 있는 마디의 가지 수는 $(n-2)$ 개
    * 그 중에 하나를 선택하면, 그 다음에 도착할 수 있는 마디의 가지 수는 $(n-3)$개
    * ...
    * 따라서 총 경로의 개수는 다음과 같음:

        $$(n-2)\times(n-3)\times\cdots\times 1= (n-2)!$$

* 이 경로의 수만 보아도 지수보다 훨씬 큼. 따라서 실용성이 전혀 없음.

### 최단경로 알고리즘 동적계획 설계 전략

#### 그래프의 인접행렬

* 마디와 마디를 잇는 이음선과 가중치의 정보를 표현하는 2차원 행렬

* 다음과 같이 정의되는 $n\times n$ 행렬 $W$로 표현할 수 있음.

$$
W[i][j] = 
\begin{cases}
\text{이음선 가중치} & \quad\text{$v_i$ 에서 $v_j$ 로의 이음선이 존재하는 경우} \\
\infty & \quad\text{$v_i$ 에서 $v_j$ 로의 이음선이 존재하지 않는 경우} \\
0 & \quad \text{$i = j$ 인 경우}
\end{cases}
$$

* 예제: 위 예제 그래프의 인접행렬

<div align="center"><img src="./images/algo03/algo03-04.png" width="250"/></div>

#### 최단경로길이 행렬

* 각 마디들 사이의 최단경로의 길이를 담은 2차원 행렬 $D$

* 예제: 위 예제 그래프의 최단경로 길이 행렬

<div align="center"><img src="./images/algo03/algo03-05.png" width="250"/></div>

#### 최단경로길이 행렬 구하기

<div align="center"><img src="./images/algo03/algo03-06.png" width="600"/></div>

#### 동적계획 전략

* 작은 입력사례 살펴보기

* $0 \le k \le n$ 를 만족하는 $k$에 대해 다음을 만족하는 2차원 행렬 $D^{(k)}$ 생성하기

\begin{align*}
D^{(k)}[i][j] &= \text{집합 $\{v_1, v_2, \dots, v_k\}$ 에 속하는 마디만을 통해서} \\
& \quad\,\text{$v_i$ 에서 $v_j$ 로 가는 최단경로의 길이}
\end{align*}

* 다음이 성립함.

\begin{align*}
D^{(0)} &= W \\
D^{(n)} &= D
\end{align*}

* 남은 과제: $0 < k \le n$ 를 만족하는 $k$에 대해, $D^{(k-1)}$ 로부터 $D^{(k)}$ 생성하기.

$$
D^{(0)} \longrightarrow D^{(1)}\longrightarrow D^{(2)}
\longrightarrow \cdots \longrightarrow D^{(n-1)}\longrightarrow D^{(n)}
$$

#### 예제

* 위 예제 그래프에 대해 $D^{(k)}[2][5]$ 계산하기

* $D^{(0)}[2][5] = W[2][5] = \infty$ 

* $D^{(1)}[2][5] = \min (D^{(0)}[2][5], \text{length}[v_2, v_1, v_5]) = \min(\infty, 14) = 14$ 

* $D^{(2)}[2][5] = D^{(1)}[2][5] = 14$

* $D^{(3)}[2][5] = D^{(2)}[2][5] = 14$

* $D^{(4)}[2][5] = \min (D^{(3)}[2][5], d^{(4)}) = \min(14, 5) = 5$

\begin{align*}
d^{(4)} &= \min(\text{length}[v_2, v_4, v_5], \text{length}[v_2, v_1, v_4, v_5], \text{length}[v_2, v_3, v_4, v_5]) \\
&= \min(5, 13, 10) \\
&= 5
\end{align*}

* $D^{(5)}[2][5] = D^{(4)}[2][5] = 5$

#### $D^{(k)}$ 의 재귀적 성질

* $D^{(k)}[i][j]$ 를 재귀적으로 정의할 수 있음.

$$D^{(k)}[i][j] = \min \big( D^{(k-1)}[i][j],\,D^{(k-1)}[i][k] + D^{(k-1)}[k][j] \big)$$

* 경우 1: $\{v_1, v_2,\dots, v_k\}$ 에 속한 마디들만을 통해서 $v_i$에서 $v_j$로 가는 최단경로가 $v_k$를 거쳐가지 않는 경우.
    
    $$D^{(k)}[i][j] = D^{(k-1)}[i][j]$$
    <br>
    
    * 예제: $D^{(5)}[1][3] = D^{(4)}[1][3] = 3$

* 경우 2: $\{v_1, v_2,\dots, v_k\}$ 에 속한 마디들만을 통해서 $v_i$에서 $v_j$로 가는 최단경로가 $v_k$를 거쳐가는 경우.
    
    $$D^{(k)}[i][j] = D^{(k-1)}[i][k] + D^{(k-1)}[k][j]$$
    <br>
    
    * 예제: $D^{(2)}[5][3] = 7 = 4 + 3 = D^{(1)}[5][2] + D^{(1)}[2][3]$

* 이유: 아래 그림 참조

<div align="center"><img src="./images/algo03/algo03-07.png" width="500"/></div>

#### 플로이드-워셜 알고리즘

* 아래 화살표 과정을 구현하는 알고리즘.
* 앞서 설명한 재귀적 성질 이용

$$
W = D^{(0)} \longrightarrow D^{(1)}\longrightarrow D^{(2)}
\longrightarrow \cdots \longrightarrow D^{(n-1)}\longrightarrow D^{(n)} = D
$$

* 입력: 마디 수가 $n$인 가중치포함 그래프. 2차원 인접행렬로 표현됨.
* 출력: 하나의 마디에서 다른 마디로 가는 최단경로의 길이를 담은 2차원 행렬.

In [19]:
def floyd_Warshall(W):
    n = len(W)
    
    # D^(0) 지정
    D = W

    # k가 0부터 (n-1)까지 이동하면서 D가 D^(1), ..., D^(n)을 차례대로 모방함.
    # 즉, D를 업데이트하는 방식을 이용하여 최종적으로 D^(n) 생성
    for k in range(0, n):
        # 행렬의 인덱스는 0부터 (n-1)까지 이동
        for i in range(0, n):
            for j in range(0, n):
                D[i][j] = min(D[i][j] , D[i][k]+ D[k][j])
    
    # 최종 완성된 D 반환
    return D

#### 예제

* 위 예제 그래프의 인접행렬은 다음과 같음.

In [17]:
# 무한에 해당하는 기호 사용
from math import inf

# inf 는 두 마디 사이에 이음선이 없음을 의미함.
W = [[0, 1, inf, 1, 5],
     [9, 0, 3, 2, inf],
     [inf, inf, 0, 4, inf],
     [inf, inf, 2, 0, 3],
     [3, inf, inf, inf, 0]]

* 플로이드-워셜 알고리즘의 결과: 앞서 살펴 본 행렬 $D$와 동일.

In [20]:
floyd_Warshall(W)

[[0, 1, 3, 1, 4],
 [8, 0, 3, 2, 5],
 [10, 11, 0, 4, 7],
 [6, 7, 2, 0, 3],
 [3, 4, 6, 4, 0]]

* 참조: [PythonTutor: 플로이드-워셜 알고리즘](http://pythontutor.com/visualize.html#code=def%20floyd_warshall%28W%29%3A%0A%20%20%20%20n%20%3D%20len%28W%29%0A%20%20%20%20%0A%20%20%20%20%23%20D%5E%280%29%20%EC%A7%80%EC%A0%95%0A%20%20%20%20D%20%3D%20W%0A%0A%20%20%20%20%23%20k%EA%B0%80%200%EB%B6%80%ED%84%B0%20%28n-1%29%EA%B9%8C%EC%A7%80%20%EC%9D%B4%EB%8F%99%ED%95%98%EB%A9%B4%EC%84%9C%20D%EA%B0%80%20D%5E%281%29,%20...,%20D%5E%28n%29%EC%9D%84%20%EC%B0%A8%EB%A1%80%EB%8C%80%EB%A1%9C%20%EB%AA%A8%EB%B0%A9%ED%95%A8.%0A%20%20%20%20%23%20%EC%A6%89,%20D%EB%A5%BC%20%EC%97%85%EB%8D%B0%EC%9D%B4%ED%8A%B8%ED%95%98%EB%8A%94%20%EB%B0%A9%EC%8B%9D%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC%20%EC%B5%9C%EC%A2%85%EC%A0%81%EC%9C%BC%EB%A1%9C%20D%5E%28n%29%20%EC%83%9D%EC%84%B1%0A%20%20%20%20for%20k%20in%20range%280,%20n%29%3A%0A%20%20%20%20%20%20%20%20%23%20%ED%96%89%EB%A0%AC%EC%9D%98%20%EC%9D%B8%EB%8D%B1%EC%8A%A4%EB%8A%94%200%EB%B6%80%ED%84%B0%20%28n-1%29%EA%B9%8C%EC%A7%80%20%EC%9D%B4%EB%8F%99%0A%20%20%20%20%20%20%20%20for%20i%20in%20range%280,%20n%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20for%20j%20in%20range%280,%20n%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20D%5Bi%5D%5Bj%5D%20%3D%20min%28D%5Bi%5D%5Bj%5D%20,%20D%5Bi%5D%5Bk%5D%2B%20D%5Bk%5D%5Bj%5D%29%0A%20%20%20%20%0A%20%20%20%20%23%20%EC%B5%9C%EC%A2%85%20%EC%99%84%EC%84%B1%EB%90%9C%20D%20%EB%B0%98%ED%99%98%0A%20%20%20%20return%20D%0A%20%20%20%20%0A%23%20%EB%AC%B4%ED%95%9C%EC%97%90%20%ED%95%B4%EB%8B%B9%ED%95%98%EB%8A%94%20%EA%B8%B0%ED%98%B8%20%EC%82%AC%EC%9A%A9%0Afrom%20math%20import%20inf%0A%0A%23%20inf%20%EB%8A%94%20%EB%91%90%20%EB%A7%88%EB%94%94%20%EC%82%AC%EC%9D%B4%EC%97%90%20%EC%9D%B4%EC%9D%8C%EC%84%A0%EC%9D%B4%20%EC%97%86%EC%9D%8C%EC%9D%84%20%EC%9D%98%EB%AF%B8%ED%95%A8.%0AW%20%3D%20%5B%5B0,%201,%20inf,%201,%205%5D,%0A%20%20%20%20%20%5B9,%200,%203,%202,%20inf%5D,%0A%20%20%20%20%20%5Binf,%20inf,%200,%204,%20inf%5D,%0A%20%20%20%20%20%5Binf,%20inf,%202,%200,%203%5D,%0A%20%20%20%20%20%5B3,%20inf,%20inf,%20inf,%200%5D%5D%0A%20%20%20%20%20%0A%20%20%20%20%20%0Afloyd_warshall%28W%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

### 최단경로 확인 알고리즘

* 이전 함수를 약간 수정하여 최단경로를 출력하는 함수 구현

#### 추가사항

* 두 마디 사이의 최단경로에 사용된 마디 중에서 가장 큰 인덱스를 기억하는 행렬 $P$

* 즉, 다음이 성립해야 함.

$$
P[i][j] =
\begin{cases}
k & \text{최단경로의 중간에 사용된 마디의 인덱스 중에서 가장 큰 값이 $k$인 경우} \\
  & \text{(아래 그림에서 사용된 $v_k$의 인덱스 $k$)}\\
  & \\
0 & \text{최단경로의 중간에 사용된 마디가 없는 경우}
\end{cases}
$$

<div align="center"><img src="./images/algo03/algo03-07.png" width="500"/></div>

* 나머지 사항은 동일함.

In [23]:
from copy import deepcopy

def floyd_warshall2(W):
    n = len(W)
    
    D = W
    
    # deepcopy를 사용하지 않으면 D에 혼란을 발생시킴
    P = deepcopy(W)
    
    # P 행렬 초기화. 모든 항목을 0으로 설정
    for i in range(n):
        for j in range(n):
            P[i][j] = 0

    # k가 0부터 (n-1)까지 이동하면서 D가 D^(1), ..., D^(n)을 차례대로 모방함.
    # 그와 함께 동시에 P 행렬도 차례대로 업데이트함.
    for k in range(0, n):
        for i in range(0, n):
            for j in range(0, n):
                if D[i][k]+ D[k][j] < D[i][j]:
                    P[i][j] = k
                    D[i][j] = D[i][k]+ D[k][j]
    
    # 최종 완성된 P 반환
    return D, P

In [ ]:
def path(W, q, r):
    if 